# Seleção de Features usando o teste chi quadrado de independência - abordagem 1

## 1. Carregando módulos e dados

In [1]:
# Manipulação dos dados
import numpy  as np
import scipy  as sp
import pandas as pd

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn           as sns
%matplotlib inline

In [2]:
# Carregando os dados
dados = pd.read_csv('../../Dados/abordagem1.csv')

In [3]:
# Visualização da tabela
dados.head(5)

,desfecho,TTO_anterior_TB_triagem_enfermeiro,Cicatriz_BCG,Idade,Sexo,Raça,Tem_companheiro,Tosse,Semanas_tosse,Expectoração,...,CAGE,Estado_civil_Casado,Estado_civil_Separado,Estado_civil_Solteiro,Estado_civil_ignorado,Estado_civil_viúvo,Fuma_Ex-fumante,Fuma_Fumante,Fuma_Jamais fumante,Fuma_ignorado
0,TB+,Sim,Sim,30,Masculino,Não branco,Não,Sim,ignorado,Não,...,Negativo,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,TB+,Sim,ignorado,53,Feminino,Branco,Não,Sim,Não,Sim,...,Negativo,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,TB+,Não,Não,84,Feminino,Branco,Não,Sim,Não,Sim,...,Negativo,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,TB+,Não,Sim,18,Feminino,Branco,Sim,Sim,Sim,Não,...,Negativo,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,TB+,Sim,ignorado,36,Masculino,Não branco,Não,Sim,Não,Sim,...,Positivo,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


## 2. Seleção de variáveis utilizando o teste do chi quadrado de independência

O teste do chi quadrado verifica se existe a relação entre duas variáveis categóricas. Dessa forma, será verificada a relação entre as variáveis presentes e o desfecho, ou seja, se o atributo possui alguma relação com o diagnóstico da doença ou não. O teste do Chi-quadrado é realizado para a verificação de independência entre duas variáveis categóricas:

- Hipótese nula $H_0$ : As variáveis são independentes.
- Hipótese alternativa $H_a$: As variáveis são dependentes

### 2.1 Abordagem conservadora: Deletar os valores ignorados

In [4]:
# Criando um novo dataframe
dados_sem_ignorados = dados.copy()

# Substituindo os valores por nulos 
dados_sem_ignorados.replace(['ignorado'],np.nan,inplace=True)

# Remover todos os casos ignorados
dados_sem_ignorados = dados_sem_ignorados.dropna(how='any')

In [5]:
from scipy.stats import chi2_contingency

# Criando um DataFrame para o armazenamento dos valores

dict_features = dict()

for feature in list(dados_sem_ignorados.columns)[1:]:
    
    # print('Tabela de Contingência entre o desfecho e {}\n'.format(feature))
    
    # Contingency table from pandas
    cross_tab = pd.crosstab(dados_sem_ignorados[feature],dados_sem_ignorados['desfecho'],margins=True)
    print(pd.crosstab(dados_sem_ignorados[feature],dados_sem_ignorados['desfecho'],margins=True))
    print()
    
    # Chi Squared Test 
    chi2, p, dof, expected =  chi2_contingency(cross_tab)
    print('Chi2: {}\tp-value: {}\tDOF: {}\nExpected_val: \n{}\n\n'.format(chi2,p,dof,expected))
    
    dict_features[feature] = [chi2,p,dof,(len(dados_sem_ignorados[feature].unique()) - 1)]

desfecho                            TB+  TB-   All
TTO_anterior_TB_triagem_enfermeiro                
Não                                 351  741  1092
Sim                                  81  234   315
All                                 432  975  1407

Chi2: 4.7486250000000005	p-value: 0.3140758065337092	DOF: 4
Expected_val: 
[[  335.28358209   756.71641791  1092.        ]
 [   96.71641791   218.28358209   315.        ]
 [  432.           975.          1407.        ]]


desfecho      TB+  TB-   All
Cicatriz_BCG                
Não           144  444   588
Sim           288  531   819
All           432  975  1407

Chi2: 18.33183431952663	p-value: 0.0010627700874393108	DOF: 4
Expected_val: 
[[  180.53731343   407.46268657   588.        ]
 [  251.46268657   567.53731343   819.        ]
 [  432.           975.          1407.        ]]


desfecho  TB+  TB-   All
Idade                   
9           0    1     1
13          1    2     3
14          5    7    12
15          7    2     9
16

In [6]:
# Criando um DataFrame a partir do dicionário criado anteriormente
chi2_dict = pd.DataFrame.from_dict(dict_features,orient='index')
chi2_dict.columns = ['chi2','pval','dof','dof_real']

In [7]:
variaveis_dependentes   = chi2_dict[chi2_dict.pval <= 0.05]
variaveis_independentes = chi2_dict[chi2_dict.pval > 0.05]

In [8]:
variaveis_dependentes

,chi2,pval,dof,dof_real
Perda_peso_10percent,111.468003,3.538947e-23,4,1
Semanas_tosse,33.925988,7.716922e-07,4,1
Sudorese_noturna,17.339842,1.660061e-03,4,1
Febre,37.044453,1.763716e-07,4,1
Estado_civil_Solteiro,35.635029,3.439649e-07,4,1
Estado_civil_Casado,10.877413,2.797683e-02,4,1
Dispnéia,11.179443,2.461965e-02,4,1
Sexo,25.371503,4.235617e-05,4,1
Delegacia_2anos,17.674704,1.428450e-03,4,1
Estado_civil_viúvo,11.737643,1.941267e-02,4,1


In [9]:
variaveis_independentes

,chi2,pval,dof,dof_real
Tosse,0.116650,0.998364,4,1
Tem_companheiro,4.401366,0.354404,4,1
Internação_hospital_2anos,0.623353,0.960434,4,1
Fuma_Fumante,1.073085,0.898519,4,1
TTO_anterior_TB_triagem_enfermeiro,4.748625,0.314076,4,1
Fuma_ignorado,0.702017,0.951080,4,1
Contato_TBP_2anos,6.904814,0.141005,4,1
Hemoptóicos,0.397907,0.982648,4,1
Raça,5.293259,0.258509,4,1
Fuma_Jamais fumante,2.781989,0.594946,4,1


In [10]:
variaveis_dependentes.T.columns

Index(['Perda_peso_10percent', 'Semanas_tosse', 'Sudorese_noturna', 'Febre',
       'Estado_civil_Solteiro', 'Estado_civil_Casado', 'Dispnéia', 'Sexo',
       'Delegacia_2anos', 'Estado_civil_viúvo', 'Cicatriz_BCG', 'Expectoração',
       'Prisão_2anos'],
      dtype='object')

### 2.2 Considerando os casos ignorados

In [26]:
# Criando um DataFrame para o armazenamento dos valores

dados_sem_ignorados = dados.copy()

dict_features_ignorados = dict()

for feature in list(dados.columns)[1:]:
        
    print('Tabela de Contingência entre o desfecho e {}\n'.format(feature))
    
    # Contingency table from pandas
    cross_tab = pd.crosstab(dados_sem_ignorados[feature],dados_sem_ignorados['desfecho'],margins=True)
    print(pd.crosstab(dados_sem_ignorados[feature],dados_sem_ignorados['desfecho'],margins=True))
    print()
    
    # Chi Squared Test 
    chi2, p, dof, expected =  chi2_contingency(cross_tab)
    print('Chi2: {}\tp-value: {}\tDOF: {}\nExpected_val: \n{}\n\n'.format(chi2,p,dof,expected))
    
    dict_features_ignorados[feature] = [chi2,p,dof,(len(dados_sem_ignorados[feature].unique()) - 1),]

Tabela de Contingência entre o desfecho e TTO_anterior_TB_triagem_enfermeiro

desfecho                            TB+   TB-   All
TTO_anterior_TB_triagem_enfermeiro                 
Não                                 668  1388  2056
Sim                                 202   453   655
ignorado                              1    15    16
All                                 871  1856  2727

Chi2: 5.508782301963685	p-value: 0.48039588021335167	DOF: 6
Expected_val: 
[[  656.68353502  1399.31646498  2056.        ]
 [  209.20608728   445.79391272   655.        ]
 [    5.1103777     10.8896223     16.        ]
 [  871.          1856.          2727.        ]]


Tabela de Contingência entre o desfecho e Cicatriz_BCG

desfecho      TB+   TB-   All
Cicatriz_BCG                 
Não           222   614   836
Sim           434   780  1214
ignorado      215   462   677
All           871  1856  2727

Chi2: 19.267143857915137	p-value: 0.0037354693229748413	DOF: 6
Expected_val: 
[[  267.01723506   568.9

In [27]:
# Criando um DataFrame a partir do dicionário criado anteriormente
chi2_dict_ignorados = pd.DataFrame.from_dict(dict_features_ignorados,orient='index')
chi2_dict_ignorados.columns = ['chi2','pval','dof','dof_real']

# Dataframe criado
chi2_dict_ignorados

,chi2,pval,dof,dof_real
Perda_de_apetite,37.388323,1.479082e-06,6,2
Fuma_Jamais fumante,8.903652,6.355336e-02,4,1
Fuma_Fumante,3.705622,4.473087e-01,4,1
Raça,10.622140,1.007806e-01,6,2
Fuma_ignorado,0.402505,9.822713e-01,4,1
CAGE,9.012802,1.728593e-01,6,2
Estado_civil_ignorado,0.541689,9.693177e-01,4,1
Tem_companheiro,9.710596,1.373805e-01,6,2
Idade,209.255984,1.286586e-02,166,82
Estado_civil_viúvo,24.049972,7.805348e-05,4,1


In [28]:
variaveis_dependentes_ignorados   = chi2_dict_ignorados[chi2_dict.pval <= 0.05]
variaveis_independentes_ignorados = chi2_dict_ignorados[chi2_dict.pval > 0.05]

In [29]:
variaveis_dependentes_ignorados

,chi2,pval,dof,dof_real
Estado_civil_viúvo,24.049972,7.805348e-05,4,1
Febre,94.595210,3.356747e-18,6,2
Estado_civil_Casado,23.673339,9.286502e-05,4,1
Delegacia_2anos,20.875915,1.931197e-03,6,2
Dispnéia,21.087264,1.769541e-03,6,2
Perda_peso_10percent,202.580545,5.356409e-41,6,2
Sexo,53.659613,8.640003e-10,6,2
Estado_civil_Solteiro,65.830393,1.719911e-13,4,1
Semanas_tosse,58.599395,8.662085e-11,6,2
Cicatriz_BCG,19.267144,3.735469e-03,6,2
